In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr
import fsspec
import salem
import xdem
import hvplot.xarray

from matplotlib import pyplot as plt
from dask.diagnostics import ProgressBar

import planetarycomputer_helper as pch

In [3]:
url_spatial = 's3://spi-pamir-cryogrid/pamir-MLpermafrost/data-inference/spatial_data-710w365s750e400n-100m-20251217.zarr/'
ds = xr.open_zarr(url_spatial).rio.write_crs('EPSG:4326')

In [4]:
with ProgressBar():
    land_cover_esa = pch.get_data(
        "esa-worldcover",
        bbox_WSEN=(71, 36.5, 75, 40),
        bands=['map'],
        resolution_m=100,
        write_safe=True,
    ).mean('time').astype('int16').compute()['map'].rename('land_cover_esa')


2025-12-18 17:01:26.393 | DEBUG    | planetarycomputer_helper.planetary_computer_helper:_log_dataset_url:149 - Opening dataset page: https://planetarycomputer.microsoft.com/dataset/esa-worldcover


2025-12-18 17:01:26.489 | DEBUG    | planetarycomputer_helper.planetary_computer_helper:get_stac_items:90 - Searching STAC catalog with query: 
	{'collections': ['esa-worldcover'], 'datetime': None, 'bbox': (71, 36.5, 75, 40), 'query': {}}
2025-12-18 17:01:26.825 | DEBUG    | planetarycomputer_helper.planetary_computer_helper:get_data:42 - Found 12 matching STAC items in Planetary Computer catalog.
2025-12-18 17:01:26.825 | DEBUG    | planetarycomputer_helper.planetary_computer_helper:stac_items_to_xarray:114 - Converting resolution from meters to degrees for EPSG:4326. 
2025-12-18 17:01:26.825 | DEBUG    | planetarycomputer_helper.planetary_computer_helper:stac_items_to_xarray:126 - Stacking collection with stackstac using props: 
	props={'resolution': 0.0008983111749910168, 'epsg': 4326, 'assets': ['map'], 'bounds_latlon': (71, 36.5, 75, 40)}
2025-12-18 17:01:26.832 | DEBUG    | planetarycomputer_helper.planetary_computer_helper:_remove_unsafe_coords:234 - Unsafe coordinates are: ['i

[########################################] | 100% Completed | 2.39 sms


In [5]:
mappings = {
    10: 5,  # Tree cover
    20: 5,  # Shrubland
    30: 5,  # Grassland
    40: 0,  # Cropland
    50: 0,  # Built-up
    60: 2,  # Bare / sparse vegetation
    70: 0,  # Snow and ice
    80: 0,  # Permanent water bodies
    90: 0,  # Herbaceous wetland
    95: 0,  # Mangroves
    100: 0   # Moss and lichen
}

In [6]:
surface_cover_index = (
    land_cover_esa
    .to_series()
    .round(-1)
    .map(mappings)
    .to_xarray()
    .rio.write_crs('EPSG:4326')
    .rolling(x=3, y=3, center=True).median()
    .rolling(x=3, y=3, center=True).median()
    .rio.reproject_match(ds)
    .astype('int16')
)

/home/renku/work/make-cryogrid-runs/.venv/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:237: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [7]:
with ProgressBar():
    glaciers = ds.glaciers.astype(bool).compute()
    rock_glaciers = ds.rock_glaciers.astype(bool).compute()
    slope = ds.slope_angle.compute()
    curvature = (
        xdem.DEM.from_xarray(ds.altitude.rio.reproject('EPSG:32643', resolution=100).rolling(x=3, y=3, center=True).mean())
        .curvature()
        .to_xarray()
        .squeeze(drop=True)
        .rio.reproject_match(ds)
        .rolling(x=3, y=3, center=True).mean()
        )

[                                        ] | 0% Completed | 117.39 ms

[########################################] | 100% Completed | 623.69 ms
[########################################] | 100% Completed | 202.61 ms
[########################################] | 100% Completed | 303.41 ms
[########################################] | 100% Completed | 303.86 ms


/home/renku/work/make-cryogrid-runs/.venv/lib/python3.12/site-packages/geoutils/raster/raster.py:1748: UserWarning: Setting default nodata -99999 to mask non-finite values found in the array, as no nodata value was defined.
  warnings.warn(


In [9]:
steep_slope_bare = (slope  > 30) & (surface_cover_index == 2)
low_slope_bare = (slope <= 30) & (surface_cover_index == 2)
flat_areas_bare = (slope <= 5) & (curvature.pipe(abs) < 0.3) & (surface_cover_index == 2)

In [12]:
sci = surface_cover_index.compute().where(~steep_slope_bare, 2)
sci = sci.where(~low_slope_bare, 3)
sci = sci.where(~flat_areas_bare, 4)
sci = sci.where(~glaciers, 0)
sci = sci.where(~rock_glaciers, 1)

In [21]:
da = sci.rename('surface_index_v2')
da.attrs = dict(
    info=(
        "Land cover ESA product downloaded from Planetary Computer, "
        "water and ice (RGI) = 0; rock glaciers = 1; bedrock = 2; "
        "talus slopes and steep bare rock = 3; "
        "flat bare surface = 4; "
        "vegetated = 5"))

ds = da.to_dataset()
ds.to_zarr('s3://spi-pamir-cryogrid/pamir-MLpermafrost/data-inference/spatial_data-710w365s750e400n-100m-20251217.zarr/', mode='a')